In [ ]:
from openai import OpenAI

client = OpenAI( api_key= "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",
                base_url="https://openrouter.ai/api/v1")

chat = client.chat.completions.create(
    
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[
         {
                "role": "system",
                "content": "Eres un experto en extracción de datos de facturas. Devuelve solo el CSV sin explicaciones ni mensajes adicionales. Si no puedes extraer datos, devuelve exactamente la palabra 'error' sin comillas.",
            },
        {"role":"user",
              "content":"Cuantas 'e' tiene la palabra enesimo"
        }     
              ]
    
)
print(chat)
print(chat.choices[0].message.content)

In [2]:
import pandas as pd
import openpyxl as px
import os
from plotly import graph_objects as go
import nbformat as nbf

In [3]:
archivos_data = [
    ["videogamesales.xlsx", "vgsales"],
    ["videogamesales.xlsx", "Total Sales by Genre"],
    ["videogamesales.xlsx", "Breakdown of Sales by Genre"],
    ["videogamesales.xlsx", "Breakdown of Sales by Year"]
]

df_archivos =pd.DataFrame(archivos_data, columns=["archivo", "sheet"])
df_archivos
hojas_df ={}
for archivo in df_archivos["archivo"]:
    if not os.path.exists(archivo):
        print(f"El archivo {archivo} no existe.")
        continue
    for hoja in df_archivos["sheet"]:
        hojas_df[hoja] = pd.read_excel(archivo, sheet_name=hoja)
        print(f"Archivo : {archivo} Hoja: {hoja}")
        print(hojas_df[hoja].head())
        print("\n")

Archivo : videogamesales.xlsx Hoja: vgsales
   Rank                      Name Platform  Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  
0     41.49     29.02      3.77         8.46  
1     29.08      3.58      6.81         0.77  
2     15.85     12.88      3.79         3.31  
3     15.75     11.01      3.28         2.96  
4     11.27      8.89     10.22         1.00  


Archivo : videogamesales.xlsx Hoja: Total Sales by Genre
  Row Labels  Sum of Sum of Sales
0     Action              1722.88
1  Adventure               234.80
2   Fighting               444.05
3       Misc          

In [21]:
hojas_df["vgsales"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16327 entries, 0 to 16326
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         16327 non-null  int64  
 1   Name         16327 non-null  object 
 2   Platform     16327 non-null  object 
 3   Year         16327 non-null  int64  
 4   Genre        16327 non-null  object 
 5   Publisher    16291 non-null  object 
 6   NA_Sales     16327 non-null  float64
 7   EU_Sales     16327 non-null  float64
 8   JP_Sales     16327 non-null  float64
 9   Other_Sales  16327 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 1.2+ MB


In [4]:
hojas_df["vgsales"].groupby(["Platform","Genre","Publisher"])[["NA_Sales","EU_Sales","JP_Sales","Other_Sales"]].sum()

NA_Sales  EU_Sales  JP_Sales  \
Platform Genre    Publisher                                                    
2600     Action   20th Century Fox Video Games      1.61      0.09       0.0   
                  Activision                        4.15      0.24       0.0   
                  Answer Software                   0.46      0.03       0.0   
                  Atari                             4.61      0.28       0.0   
                  Avalon Interactive                0.15      0.01       0.0   
...                                                  ...       ...       ...   
XOne     Sports   Ubisoft                           0.00      0.02       0.0   
                  Unknown                           0.02      0.00       0.0   
         Strategy Activision                        0.11      0.04       0.0   
                  Microsoft Game Studios            0.14      0.05       0.0   
                  Take-Two Interactive              0.01      0.01       0.0   

                                                Other_Sales  
Platform Genre    Publisher                                  
2600     Action   20th Century Fox Video Games         0.01  
                  Activision                           0.03  
                  Answer Software                      0.01  
                  Atari                                0.05  
                  Avalon Interactive                   0.00  
...                                                     ...  
XOne     Sports   Ubisoft                              0.00  
                  Unknown                              0.00  
         Strategy Activision                           0.01  
                  Microsoft Game Studios               0.02  
                  Take-Two Interactive                 0.00  

[4979 rows x 4 columns]

In [5]:
pd.options.display.float_format = '{:.2f}'.format
ventas_agrupadas = hojas_df["vgsales"].groupby(['Platform', 'Genre', 'Publisher'])[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum().reset_index()
print(ventas_agrupadas.head())

  Platform   Genre                     Publisher  NA_Sales  EU_Sales  \
0     2600  Action  20th Century Fox Video Games      1.61      0.09   
1     2600  Action                    Activision      4.15      0.24   
2     2600  Action               Answer Software      0.46      0.03   
3     2600  Action                         Atari      4.61      0.28   
4     2600  Action            Avalon Interactive      0.15      0.01   

   JP_Sales  Other_Sales  
0      0.00         0.01  
1      0.00         0.03  
2      0.00         0.01  
3      0.00         0.05  
4      0.00         0.00  


In [6]:
import plotly.express as px

In [7]:
grafica = px.histogram(hojas_df["vgsales"], x="Platform", y="NA_Sales", color="Genre", title="Ventas por Plataforma y Género en NA")
grafica.show()
grafica.write_html("grafica.html")

In [8]:
lista_ventas = ["NA_Sales", "EU_Sales", "JP_Sales"]
for columna in lista_ventas:
    grafica = px.histogram(hojas_df["vgsales"],
                            x="Platform", y=columna,
                            color="Genre",
                            color_discrete_sequence=px.colors.qualitative.Plotly,
                            title=f"Ventas por Plataforma y Género en {columna}",
                            labels={columna: f"Ventas en {columna.replace('_', ' ').upper()}"} # Cambia el título del eje y
                            )
    grafica.update_layout(
        yaxis_title=f"Ventas en {columna.replace('_', ' ').upper()}"  # Establece el título del eje y
    )
    grafica.show()
    grafica.write_html(f"grafica_{columna}.html")

In [11]:
# Get the vgsales DataFrame
df = hojas_df["vgsales"].copy()

# 1. First ensure we have numeric values for sales columns
sales_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
for col in sales_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
# 2. Create Global_Sales if it doesn't exist
if 'Global_Sales' not in df.columns:
    df['Global_Sales'] = df[sales_cols].sum(axis=1)

# 3. Fix the correlation heatmap - ensure we're using the correct columns
sales_corr = df[sales_cols].corr()
fig5 = px.imshow(sales_corr,
                text_auto=True,
                aspect="auto",
                title='Sales Correlation Between Regions',
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1,
                labels=dict(color="Correlation"))
fig5.update_xaxes(side="bottom")

# 4. For the top games visualization
top_games = df.nlargest(10, 'Global_Sales')
fig6 = px.bar(top_games, 
             x='Name', 
             y=sales_cols,
             title='Top 10 Games by Region Sales (in millions)',
             barmode='stack',
             labels={'value': 'Sales (millions)', 'variable': 'Region'})
fig6.update_layout(xaxis_tickangle=-45)

fig5.show()
fig6.show()